# 划分训练集和测试集

同济子豪兄 https://space.bilibili.com/1900783

代码运行[云GPU平台](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)

2022-7-22

## 导入工具包

In [17]:
import os
import shutil
import random
import pandas as pd

## 获得所有类别名称

In [18]:
# 指定数据集路径
dataset_path = 'fruit81_full'

In [19]:
dataset_name = dataset_path.split('_')[0]
print('数据集', dataset_name)

数据集 fruit81


In [20]:
classes = os.listdir(dataset_path)

In [21]:
len(classes)

81

In [22]:
classes

['莲雾',
 '黄桃',
 '圣女果',
 '芒果',
 '菠萝',
 '石榴',
 '桑葚',
 '沃柑',
 '山竹',
 '血橙',
 '人参果',
 '羊角蜜',
 '菠萝莓',
 '猕猴桃',
 '哈密瓜',
 '山楂',
 '黑莓',
 '沙棘',
 '柚子',
 '脐橙',
 '杨桃',
 '甘蔗',
 '车厘子',
 '杨梅',
 '蛇皮果',
 '西柚',
 '百香果',
 '牛油果',
 '无花果',
 '草莓',
 '白兰瓜',
 '椰子',
 '西瓜',
 '蓝莓',
 '桂圆',
 '佛手瓜',
 '葡萄-红',
 '毛丹',
 '酸角',
 '番石榴-红',
 '荔枝',
 '香橼',
 '香蕉',
 '水蜜桃',
 '柠檬',
 '甜瓜-白',
 '胡萝卜',
 '橘子',
 '白心火龙果',
 '梨',
 '蟠桃',
 '甜瓜-伊丽莎白',
 '葡萄-白',
 '杏',
 '砂糖橘',
 '芦柑',
 '樱桃',
 '枣',
 '榴莲',
 '粑粑柑',
 '枇杷',
 '西红柿',
 '甜瓜-金',
 '柿子',
 '菠萝蜜',
 '甜瓜-绿',
 '羊奶果',
 '油桃',
 '树莓',
 '西梅',
 '红心火龙果',
 '沙果',
 '青苹果',
 '番石榴-百',
 '白萝卜',
 '李子',
 '金桔',
 '红苹果',
 '青柠',
 '木瓜',
 '腰果']

## 创建训练集文件夹和测试集文件夹

In [23]:
# 创建 train 文件夹
os.mkdir(os.path.join(dataset_path, 'train'))

# 创建 test 文件夹
os.mkdir(os.path.join(dataset_path, 'val'))

# 在 train 和 test 文件夹中创建各类别子文件夹
for fruit in classes:
    os.mkdir(os.path.join(dataset_path, 'train', fruit))
    os.mkdir(os.path.join(dataset_path, 'val', fruit))

## 划分训练集、测试集，移动文件

In [24]:
test_frac = 0.2  # 测试集比例
random.seed(123) # 随机数种子，便于复现

In [25]:
df = pd.DataFrame()

print('{:^18} {:^18} {:^18}'.format('类别', '训练集数据个数', '测试集数据个数'))

for fruit in classes: # 遍历每个类别

    # 读取该类别的所有图像文件名
    old_dir = os.path.join(dataset_path, fruit)
    images_filename = os.listdir(old_dir)
    random.shuffle(images_filename) # 随机打乱

    # 划分训练集和测试集
    testset_numer = int(len(images_filename) * test_frac) # 测试集图像个数
    testset_images = images_filename[:testset_numer]      # 获取拟移动至 test 目录的测试集图像文件名
    trainset_images = images_filename[testset_numer:]     # 获取拟移动至 train 目录的训练集图像文件名

    # 移动图像至 test 目录
    for image in testset_images:
        old_img_path = os.path.join(dataset_path, fruit, image)         # 获取原始文件路径
        new_test_path = os.path.join(dataset_path, 'val', fruit, image) # 获取 test 目录的新文件路径
        shutil.move(old_img_path, new_test_path) # 移动文件

    # 移动图像至 train 目录
    for image in trainset_images:
        old_img_path = os.path.join(dataset_path, fruit, image)           # 获取原始文件路径
        new_train_path = os.path.join(dataset_path, 'train', fruit, image) # 获取 train 目录的新文件路径
        shutil.move(old_img_path, new_train_path) # 移动文件
    
    # 删除旧文件夹
    assert len(os.listdir(old_dir)) == 0 # 确保旧文件夹中的所有图像都被移动走
    shutil.rmtree(old_dir) # 删除文件夹
    
    # 工整地输出每一类别的数据个数
    print('{:^18} {:^18} {:^18}'.format(fruit, len(trainset_images), len(testset_images)))
    
    # 保存到表格中
    df = df.append({'class':fruit, 'trainset':len(trainset_images), 'testset':len(testset_images)}, ignore_index=True)

# 重命名数据集文件夹
shutil.move(dataset_path, dataset_name+'_split')

# 数据集各类别数量统计表格，导出为 csv 文件
df['total'] = df['trainset'] + df['testset']
df.to_csv('数据量统计.csv', index=False)

        类别              训练集数据个数            测试集数据个数      
        莲雾                156                 39        
        黄桃                155                 38        
       圣女果                158                 39        
        芒果                139                 34        
        菠萝                158                 39        
        石榴                153                 38        
        桑葚                156                 39        
        沃柑                159                 39        
        山竹                152                 38        
        血橙                150                 37        
       人参果                146                 36        
       羊角蜜                157                 39        
       菠萝莓                 91                 22        
       猕猴桃                158                 39        
       哈密瓜                157                 39        
        山楂                159                 39        
        黑莓                150  

In [26]:
df

,class,trainset,testset,total
0,莲雾,156.0,39.0,195.0
1,黄桃,155.0,38.0,193.0
2,圣女果,158.0,39.0,197.0
3,芒果,139.0,34.0,173.0
4,菠萝,158.0,39.0,197.0
...,...,...,...,...
76,金桔,145.0,36.0,181.0
77,红苹果,142.0,35.0,177.0
78,青柠,119.0,29.0,148.0
79,木瓜,156.0,38.0,194.0


## 查看文件目录结构

In [16]:
!sudo snap install tree

In [27]:
!tree fruit81_split -L 2

fruit81_split
├── train
│   ├── 人参果
│   ├── 佛手瓜
│   ├── 哈密瓜
│   ├── 圣女果
│   ├── 山楂
│   ├── 山竹
│   ├── 无花果
│   ├── 木瓜
│   ├── 李子
│   ├── 杏
│   ├── 杨桃
│   ├── 杨梅
│   ├── 枇杷
│   ├── 枣
│   ├── 柚子
│   ├── 柠檬
│   ├── 柿子
│   ├── 树莓
│   ├── 桂圆
│   ├── 桑葚
│   ├── 梨
│   ├── 椰子
│   ├── 榴莲
│   ├── 樱桃
│   ├── 橘子
│   ├── 毛丹
│   ├── 水蜜桃
│   ├── 沃柑
│   ├── 沙果
│   ├── 沙棘
│   ├── 油桃
│   ├── 牛油果
│   ├── 猕猴桃
│   ├── 甘蔗
│   ├── 甜瓜-伊丽莎白
│   ├── 甜瓜-白
│   ├── 甜瓜-绿
│   ├── 甜瓜-金
│   ├── 番石榴-百
│   ├── 番石榴-红
│   ├── 白兰瓜
│   ├── 白心火龙果
│   ├── 白萝卜
│   ├── 百香果
│   ├── 石榴
│   ├── 砂糖橘
│   ├── 粑粑柑
│   ├── 红心火龙果
│   ├── 红苹果
│   ├── 羊奶果
│   ├── 羊角蜜
│   ├── 胡萝卜
│   ├── 脐橙
│   ├── 腰果
│   ├── 芒果
│   ├── 芦柑
│   ├── 草莓
│   ├── 荔枝
│   ├── 莲雾
│   ├── 菠萝
│   ├── 菠萝莓
│   ├── 菠萝蜜
│   ├── 葡萄-白
│   ├── 葡萄-红
│   ├── 蓝莓
│   ├── 蛇皮果
│   ├── 蟠桃
│   ├── 血橙
│   ├── 西柚
│   ├── 西梅
│   ├── 西瓜
│   ├── 西红柿
│   ├── 车厘子
│   ├── 酸角
│   ├── 金桔
│   ├── 青柠
│   ├── 青苹果
│   ├── 香橼
│   ├── 香蕉
│   ├── 黄桃
│   └── 黑莓
└── val
    ├── 人参果
    ├── 佛手瓜
    ├─